In [1]:
from faker import Faker 

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import random

In [5]:
fake=Faker()

In [6]:
def generate_dummy_data(num_products=10):
    data = []
    for _ in range(num_products):
        brand = random.choice(["Apple", "Motorola", "Realme", "Samsung", "Nokia", "Vivo", "Oppo", "Reno", "Techno"])
        if brand == "Apple":
            operating_system = "iOS"
        else:
            operating_system = random.choice(["Android", "iOS"])
        
        price = round(random.uniform(1000, 200000), 2)
        screen_size = round(random.uniform(4, 7), 1)
        storage = random.choice([32, 64, 128, 256, 512])
        ram = random.choice([2, 4, 6, 8, 12])
        camera = fake.random_element(elements=('12MP', '16MP', '20MP', '24MP'))
        
        data.append((brand, price, screen_size, storage, ram, camera))
    
    return data




Shape: (300, 6)
        Brand      Price  Screen_Size  Storage  Ram Camera
0        Reno   46199.00          6.9      256    2   24MP
1        Vivo  193616.67          6.4      512    2   16MP
2       Nokia   45415.99          4.5       64    6   20MP
3        Oppo  157629.63          5.5       64    6   12MP
4       Apple   87800.42          5.4       32   12   16MP
..        ...        ...          ...      ...  ...    ...
295  Motorola   65054.47          6.6      128    2   20MP
296      Oppo  135385.74          4.4      128    8   20MP
297      Vivo   84803.96          7.0      256    8   12MP
298   Samsung  134061.42          6.4      512    2   20MP
299   Samsung  148597.17          6.4      128    8   24MP

[300 rows x 6 columns]


In [ ]:
dummy_data = generate_dummy_data(300)
df = pd.DataFrame(dummy_data, columns=['Brand','Price', 'Screen_Size', 'Storage', 'Ram', 'Camera'])
print("Shape:", df.shape)
print(df)


In [7]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Aryan@#123",
    database="phone_products"
)

# Create a cursor object
cursor = conn.cursor()

In [8]:

for row in df.itertuples():
    query = "INSERT INTO product (brand, price, screen_size, storage, ram, camera) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row.Brand, row.Price, row.Screen_Size, row.Storage, row.Ram, row.Camera)
    cursor.execute(query, values)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


In [9]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Aryan@#123",
    database="phone_products"
)
# Query to retrieve data
query = "SELECT * FROM product"

# Load data into a DataFrame
df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Display the first few rows of the DataFrame to verify the data
print(df.head())

   id    brand      price  screen_size  storage  ram camera
0   1     Reno  196595.27          5.6       32    4   16MP
1   2     Reno  198823.18          6.5       32    4   24MP
2   3  Samsung  131738.06          4.6      256    2   24MP
3   4    Apple   39000.91          4.7       64    2   16MP
4   5   Techno   78409.59          5.0       32    8   12MP


C:\Users\aryan\AppData\Local\Temp\ipykernel_14388\1368689459.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Split the data into features (X) and target variable (y)
X = df[['screen_size', 'storage', 'ram']]
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose a regression algorithm and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)  # Corrected the non-printable character


Mean Squared Error: 3443590583.0438013


In [11]:
import tkinter as tk
from tkinter import messagebox

# Create a Tkinter window
window = tk.Tk()
window.title("Product Price Predictor")

# Function to make price prediction
def predict_price():
    # Get input values from entry fields
    screen_size = float(screen_size_entry.get())
    storage = int(storage_entry.get())
    ram = int(ram_entry.get())
    
    # Make prediction using the trained model
    predicted_price = model.predict([[screen_size, storage, ram]])
    
    # Display the predicted price
    messagebox.showinfo("Price Prediction", f"Predicted Price: Rs{predicted_price[0]:.2f}")

# GUI elements
screen_size_label = tk.Label(window, text="Screen Size (inches):")
screen_size_label.grid(row=0, column=0)

screen_size_entry = tk.Entry(window)
screen_size_entry.grid(row=0, column=1)

storage_label = tk.Label(window, text="Storage (GB):")
storage_label.grid(row=1, column=0)

storage_entry = tk.Entry(window)
storage_entry.grid(row=1, column=1)

ram_label = tk.Label(window, text="RAM (GB):")
ram_label.grid(row=2, column=0)

ram_entry = tk.Entry(window)
ram_entry.grid(row=2, column=1)

predict_button = tk.Button(window, text="Predict Price", command=predict_price)
predict_button.grid(row=3, column=0, columnspan=2)

# Run the Tkinter event loop
window.mainloop()